<div style="font-size:18pt; padding-top:20px; text-align:center"> Обработка потоковых данных с <b>Spark Streaming</b> и <b>Kafka</b></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Запуск приложения на Cloudera</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#1a">Установка Kafka на машине с Cloudera'ой</a></li>
                <li><a href="#1b">Запуск кода в Spark Streaming</a></li>
                <li><a href="#1b">Проверка работоспособности</a></li>
            </ol>
        </li>
        <li><a href="#2">Источники</a>
        </li>
    </ol>
</div>

<p>Подключение стилей для оформления тетради</p>

In [1]:
from IPython.core.display import HTML
HTML("<link rel='stylesheet' href='css/style.css'>")

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Запуск приложения на Cloudera</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name="1a"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Установка Kafka на машине с Cloudera'ой
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

<div class="msg-block msg-imp">
  <span class="msg-text-imp"></span>
  <p>
      Проверьте, что на машине с Cloudera'ой установлен и работает Zookeeper
  </p>
  <div class="code-block"><p class="code-font"><span class="code-key">zookeeper-server</span> status</p></div>
</div>

<p><b>Вариант 1</b></p>

In [ ]:
Cloudera Manager -> Parcels -> Kafka -> Download -> Distribute -> Activate

In [ ]:
Clusters -> Cloudera QuickStart -> Actions -> Add Service -> Kafka
If the "runOutMemory" error appears, change kafka configuration property "Java Heap Size of Broker" to 250MB
If failed, check whether zookepper is run

<p><b>Вариант 2</b></p>

<p>Установите <span class="code-font code-text-key">Kafka</span>-клиент</p>

In [ ]:
sudo yum install kafka

<p>Установите <span class="code-font code-text-key">Kafka</span>-сервер</p>

In [ ]:
sudo yum install kafka-server

В файле <span class="code-font">/etc/kafka/conf/server.properties</span> проверьте параметры

In [ ]:
broker.id=0
listeners=PLAINTEXT://:9092
zookeeper.connect=localhost:2181

<p>Запустите <span class="code-font code-text-key">Kafka</span>-сервер</p>

In [ ]:
sudo service kafka-server start

<p>или</p>

In [ ]:
sudo /usr/lib/kafka/bin/kafka-server-start.sh /etc/kafka/conf/server.properties

<p>Запустите клиента <span class="code-font code-text-key">Zookeeper</span></p>

In [ ]:
zookeeper-client

<p>Проверьте, что брокеры <span class="code-font code-text-key">Kafka</span> зарегистрированы в <span class="code-font code-text-key">Zookeeper</span></p>

In [ ]:
#Список всех брокеров
ls /brokers/ids

#broker.id=0
get /brokers/ids/0

#выход
quit

<div class="msg-block msg-info">
  <span class="msg-text-info">Замечание! </span> <p>Про установку и настройку Kafka на Cloudera смотрите по ссылке ниже</p>
  <a href="http://www.cloudera.com/documentation/kafka/latest/topics/kafka_installing.html#concept_ngx_4l4_4r">http://www.cloudera.com/documentation/kafka/latest/topics/kafka_installing.html#concept_ngx_4l4_4r</a>
</div>

<p>Создайте топик в <span class="code-font code-text-key">Kafka</span></p>

In [ ]:
kafka-topics --create --zookeeper localhost:2181 --topic wordCount --partition 1 --replication-factor 1

<p>Проверьте, что топик зарегистрирован в <span class="code-font code-text-key">Zookeeper</span></p>

In [ ]:
zookeeper-client

ls /brokers/topics

<p>Проверка работы</p>

In [ ]:
kafka-console-consumer --zookeeper localhost:2181 --topic tweets-kafka --from-beginning

In [ ]:
kafka-console-producer --broker-list localhost:9092 -topic wordCount

<a name="1b"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Запуск кода в Spark Streaming
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1a">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#1c">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Сохраните исходный код в файле</p>

In [3]:
# Полный путь, где будет храниться файл с исходным кодом
spStreamKafka = "/YOUR_PATH/wordcount_streaming.py"

In [ ]:
# Сохраняем исходный код в файле wordcount_streaming.py
%writefile $spStreamKafka

# -*- coding: utf-8 -*-

import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

zk_server = "localhost:2181" # сервер Zookeeper
topic = "wordCount" # топик

# Функция для обновления значений количества слов
def updateTotalCount(currentCount, countState):
    if countState is None:
        countState = 0
    return sum(currentCount, countState)

# Создаем Spark Context
sc = SparkContext(appName="KafkaWordCount")

sc.setLogLevel("OFF")

# Создаем Streaming Context
ssc = StreamingContext(sc, 10)

# Объявляем checkpoint и указываем директорию в HDFS, где будут храниться значения
ssc.checkpoint("tmp_spark_streaming1")

# Создаем подписчика на поток от Kafka c топиком topic = "wordCount"
kafka_stream = KafkaUtils.createStream(ssc, zk_server, "spark-streaming-consumer", {topic: 1})

# Трансформируем мини-batch 
lines = kafka_stream.map(lambda x: x[1])

# Подсчитывем количество слов для мини-batch
counts = lines.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda x1, x2: x1 + x2)

# Обновляем значения количества слов с учетом нового мини-batch
total_counts = counts.updateStateByKey(updateTotalCount)

# Выводим текущий результат
total_counts.pprint()

# Запускаем Spark Streaming
ssc.start()

# Ожидаем остановку
ssc.awaitTermination()

<p>Запустите код в <span class="code-font code-text-key">Spark Streaming</span></p>

In [ ]:
spark-submit \
--master local[2] \
#--conf "spark.dynamicAllocation.enabled=false" \
#--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 \
/home/cloudera/wordcount_streaming.py

<a name="1c"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            c. Проверка работоспособности
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1b">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#2">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Откройте новый терминал и запустите  <span class="code-font">producer</span> <span class="code-font code-text-key">Kafka</span></p>

In [ ]:
kafka-console-producer --broker-list localhost:9092 -topic wordCount

<p>Введите в терминале слова и отслеживайте изменения в терминале, в котором запущена программа под <span class="code-font code-text-key">Spark Streaming</span></p>

<p>Если происходит подсчет слов, значит всё работает</p>

<div class="msg-block msg-info">
  <span class="msg-text-info">Замечание! </span> <p>Про запуск и проверку работы программы Spark Streaming и Kafka  смотрите по ссылкам ниже</p>
  <a href="https://www.cloudera.com/documentation/enterprise/5-8-x/topics/spark_streaming.html">https://www.cloudera.com/documentation/enterprise/5-8-x/topics/spark_streaming.html</a><br>
<a href="https://www.hugopicado.com/2016/09/17/spark-stateful-streaming-with-python.html">https://www.hugopicado.com/2016/09/17/spark-stateful-streaming-with-python.html</a><br>
</div>

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
http://www.cloudera.com/documentation/kafka/latest/topics/kafka_installing.html
https://www.cloudera.com/documentation/enterprise/5-8-x/topics/spark_streaming.html
https://www.hugopicado.com/2016/09/17/spark-stateful-streaming-with-python.html